In [1]:
from mycnn.lenet import *
import tensorflow as tf
import numpy as np

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# reshape 784 to 28x28x1
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# pad image from 28x28x1 to 32x32x1
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)))
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)))
print(x_train.shape)
print(x_test.shape)

# convert to float and normalize
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.0
x_test = x_test / 255.0

# one-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

(60000, 32, 32, 1)
(10000, 32, 32, 1)


In [3]:
lenet5 = LeNet5()
lenet5.build()
lenet5.summary()

Model: "cnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d (AveragePo (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_1 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               48120   

In [4]:
lenet5.setup_training(
    'log_lenet5', 10, 100,
    optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.0, decay=0.0),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Create new the directory for training log !!!!!
******************************
Training new model...
******************************
checkpoint_path : log_lenet5\weights
best_checkpoint : log_lenet5\weights.h5
nb_epoch        : 10
initial_epoch   : 0
batch_size      : 100
optimizer       : <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x000001E3E94C8948>
loss            : categorical_crossentropy
metrics         : ['accuracy']


In [5]:
lenet5.train(x_train[:50000], y_train[:50000], x_train[50000:], y_train[50000:])

Epoch 1/10
500/500 [==============================] - 6s 8ms/step - loss: 0.7265 - accuracy: 0.7903 - val_loss: 0.1889 - val_accuracy: 0.9457

Epoch 00001: val_loss improved from inf to 0.18889, saving model to log_lenet5\weights.h5
Epoch 2/10
500/500 [==============================] - 3s 6ms/step - loss: 0.1792 - accuracy: 0.9458 - val_loss: 0.1150 - val_accuracy: 0.9665

Epoch 00002: val_loss improved from 0.18889 to 0.11495, saving model to log_lenet5\weights.h5
Epoch 3/10
500/500 [==============================] - 3s 6ms/step - loss: 0.1167 - accuracy: 0.9645 - val_loss: 0.0925 - val_accuracy: 0.9739

Epoch 00003: val_loss improved from 0.11495 to 0.09245, saving model to log_lenet5\weights.h5
Epoch 4/10
500/500 [==============================] - 3s 6ms/step - loss: 0.0831 - accuracy: 0.9758 - val_loss: 0.0730 - val_accuracy: 0.9786

Epoch 00004: val_loss improved from 0.09245 to 0.07296, saving model to log_lenet5\weights.h5
Epoch 5/10
500/500 [==============================] - 3s